----------------------------
## **Imports et packages**
----------------------------

In [1]:
import pandas as pd
import os
import json
import random
import warnings
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.inspection import permutation_importance

import mlflow
import mlflow.sklearn
import joblib

warnings.filterwarnings("ignore")
from utils import save_residual_plot, save_pred_vs_true_plot

In [2]:
PROJECT_ROOT = Path.cwd().parent
INPUTS_DIR = PROJECT_ROOT / "inputs"
MODEL_DIR = PROJECT_ROOT / "model"
MODEL_DIR.mkdir(exist_ok=True)

----------------------------
## **Reproductibilité et chargement des données :**
----------------------------

In [3]:

SEED = 11
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

----------------------------
## **Config**
----------------------------

In [5]:
@dataclass
class Config:
    data_path: str = "data/clean_data.csv"
    target: str = "hg/ha_yield"
    experiment_name: str = "crop-yield-prediction"
    tracking_uri: str = "http://127.0.0.1:5000"   # tracking local
    split_mode: str = "time"              # "time" ou "random"
    time_split_year: int = 2010           # si split temporel: train <= 2008, test >= 2009
    n_iter_search: int = 50              # nb essais RandomizedSearch
    cv_folds: int = 5
    # Variables proxi
    COEF_IRRIGATION_HG_HA = 12000  # l'irrigation augmente le rendement de 12000 hg/ha
    COEF_FERTILIZATION_HG_HA = 15000 # la fertilisation augmente le rendement de 15000 hg/ha


CFG = Config()

In [6]:
# ================== 1. CHARGEMENT DES DONNÉES ==================
print("\n CHARGEMENT DES DONNÉES")
df = pd.read_csv(INPUTS_DIR /"clean_data.csv")
print(f"✓ Dataset chargé: {df.shape[0]} observations, {df.shape[1]} colonnes")


 CHARGEMENT DES DONNÉES
✓ Dataset chargé: 28242 observations, 9 colonnes


----------------------------
## **Preprocessing**
----------------------------

In [11]:
df.head()

,area,item,year,avg_rain_mm,pesticides_tonnes,avg_temp,area_code,hg/ha_yield,item_code
0,albania,maize,1990,1485.0,121.0,16.37,3,36613.00,56
1,albania,potatoes,1990,1485.0,121.0,16.37,3,69120.22,116
2,albania,"rice, paddy",1990,1485.0,121.0,16.37,3,23489.15,27
3,albania,sorghum,1990,1485.0,121.0,16.37,3,12383.34,83
4,albania,soybeans,1990,1485.0,121.0,16.37,3,7000.00,236


**Pipeline de preprocessing :**

In [13]:
categorical = ["area", "item"]
numeric = ["year", "avg_rain_mm", "pesticides_tonnes", "avg_temp"]

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), # on remplace les valeurs manquantes par most-frequent
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ])

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ])

preprocessor = ColumnTransformer(transformers=[
    ("cat", cat_pipe, categorical),
    ("num", num_pipe, numeric),
    ], remainder="drop" # suppression des autres colonnes.
    )


**Split temporel :**

In [14]:
feature_cols = ["area", "item", "year", "avg_rain_mm", "pesticides_tonnes", "avg_temp"] #selection des features

X = df[feature_cols]
y = df[CFG.target] # target est défini dans CFG

train_mask = df["year"] < CFG.time_split_year
test_mask =  df["year"] >= CFG.time_split_year # ou peut également faire ~train_mask

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print(f"✓ Split effectué: Train={len(X_train)}, Test={len(X_test)}")

✓ Split effectué: Train=23233, Test=5009


feature_cols = ["area", "item", "year", "avg_rain_mm", 
                "pesticides_tonnes", "avg_temp"]

Split temporel
train_df = df[df["year"] < CFG.time_split_year]
test_df  = df[df["year"] >= CFG.time_split_year]

Conserver uniquement les pays présents dans les deux périodes
common_areas = set(train_df["area"]).intersection(test_df["area"])

train_df = train_df[train_df["area"].isin(common_areas)]
test_df  = test_df[test_df["area"].isin(common_areas)]

Création des jeux
X_train = train_df[feature_cols]
y_train = train_df[CFG.target]

X_test = test_df[feature_cols]
y_test = test_df[CFG.target]

print(f"✓ Train={len(X_train)}, Test={len(X_test)}")
print(f"✓ Areas communes: {len(common_areas)}")

**Définition des modèles :**

In [16]:
print("\nDÉFINITION DES MODÈLES À COMPARER")

models = {
    # Dummy
    "Dummy": Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", DummyRegressor(strategy="mean"))
    ]),

    # Ridge
    "Ridge": Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", Ridge(random_state=SEED))
    ]),

    # Random Forest
    "RF": Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", RandomForestRegressor(random_state=SEED, n_jobs=-1))
    ]),

    # Histogram Gradient Boosting ( plus rapide et performant )
    "HGB": Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", HistGradientBoostingRegressor(random_state=SEED))
    ]),

    # XGBoost
    "XgBoost": Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", XGBRegressor(random_state=SEED, n_jobs=-1))
    ])
}

print(f"✓ {len(models)} modèles configurés :")
for name in models.keys():
    print(f"  - {name}")


DÉFINITION DES MODÈLES À COMPARER
✓ 5 modèles configurés :
  - Dummy
  - Ridge
  - RF
  - HGB
  - XgBoost


-------------------------------
## **Entrainement des modèles**
-------------------------------

In [ ]:
# ==================  COMPARAISON DES MODÈLES ==================
print("\n COMPARAISON DES MODÈLES ")

if mlflow.active_run() is not None:
    mlflow.end_run()
    
experiment_name = CFG.experiment_name
# Vérifie si l'experiment existe
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    mlflow.create_experiment(experiment_name)

# Puis set l'experiment
mlflow.set_experiment(experiment_name)
mlflow.set_tracking_uri(CFG.tracking_uri)

results = []

scoring = {
    "R2": "r2",
    "RMSE": "neg_mean_squared_error",
    "MAE": "neg_mean_absolute_error"
}

with mlflow.start_run(run_name=f"benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as parent:
    mlflow.set_tag("split_mode", CFG.split_mode)
    mlflow.set_tag("seed", SEED)
    mlflow.log_params({
        "data_path": CFG.data_path,
        "time_split_year": CFG.time_split_year,
        "cv_folds": CFG.cv_folds,
        "n_iter_search": CFG.n_iter_search
        })
    
    for model_name, model in models.items():
         with mlflow.start_run(run_name=f"model_{model_name}", nested=True):
            
            print(f"\n  Analyse du modèle : {model_name}...")

            #Log des paramètres
            mlflow.log_param("model_type", model_name)
            mlflow.log_param("random_seed", SEED)
            mlflow.log_param("train_size", len(X_train))
            mlflow.log_param("test_size", len(X_test))


            # Cross-validation sur le train set
            
            cv_results = cross_validate(
                model, X_train, y_train,
                cv=CFG.cv_folds, #5 folds
                scoring=scoring,
                return_train_score=True)

            # Moyenne et écart-type des métriques CV
            cv_r2_mean   = cv_results['test_R2'].mean()
            cv_r2_std    = cv_results['test_R2'].std()
            cv_rmse_mean = np.sqrt(-cv_results['test_RMSE']).mean()
            cv_mae_mean  = -cv_results['test_MAE'].mean()

            # Entraînement
            start = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start
            mlflow.log_metric("train_time_sec", train_time)
            mlflow.log_params(model.get_params())

        
            # Prédictions sur train et test
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
        
            # Métriques train/test
            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)
            mae_train = mean_absolute_error(y_train, y_pred_train)
            mae_test = mean_absolute_error(y_test, y_pred_test)

            # Calcul de l'Overfit = différence R² train/test
            overfit = r2_train - r2_test

            # Log des métriques
            mlflow.log_metrics({
                "R2_Train": r2_train,
                "R2_Test": r2_test,
                "CV_R2_mean": cv_r2_mean,
                "CV_R2_std": cv_r2_std,
                "RMSE_Train": rmse_train,
                "RMSE_Test": rmse_test,
                "CV_RMSE_Mean": cv_rmse_mean,
                "MAE_Train": mae_train,
                "MAE_Test": mae_test,
                "CV_MAE_Mean": cv_mae_mean,
                "Overfit": overfit
                })
            
            
            # DIAGNOSTICS VISUELS

            # version précédente : 
            os.makedirs("artifacts", exist_ok=True)
            rv_path = f"artifacts/residuals_{model_name}.png"
            pv_path = f"artifacts/pred_vs_true_{model_name}.png"

            save_residual_plot(y_test.values, y_pred_test, rv_path)
            save_pred_vs_true_plot(y_test.values, y_pred_test, pv_path)
            mlflow.log_artifact(rv_path)
            mlflow.log_artifact(pv_path)


            # Sauvegarde du modèle
            mlflow.sklearn.log_model(model, f"model_{model_name}")
            
            # Stockage des résultats
            results.append({
                'Model': model_name,
                'R2_Train': r2_train,
                'R2_Test': r2_test,
                'CV_R2_Mean': cv_r2_mean,
                'CV_R2_Std': cv_r2_std,
                'RMSE_Train': rmse_train,
                'RMSE_Test': rmse_test,
                'CV_RMSE_Mean': cv_rmse_mean,
                'MAE_Train': mae_train,
                'MAE_Test': mae_test,
                'CV_MAE_Mean': cv_mae_mean,
                'Overfit': overfit
                })
            
            # Création du tableau comparatif des modèles

            results_df = pd.DataFrame(results)
            results_df = results_df.sort_values('R2_Test', ascending=False)
            results_df.to_csv("artifacts/feature_importance.csv", index=False)
            mlflow.log_artifact("artifacts/feature_importance.csv")



 COMPARAISON DES MODÈLES 
🏃 View run puzzled-pig-757 at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/0142e476401d4435b708067cedbf85de
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

  Analyse du modèle : Dummy...


2026/01/16 17:57:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run model_Dummy at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/8c827b66430a41a5890d02977201c98f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

  Analyse du modèle : Ridge...


2026/01/16 17:57:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run model_Ridge at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/24fb3fc81f444f45b5c421bf47043e0f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

  Analyse du modèle : RF...


...

🏃 View run model_RF at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/345821177aef4f78bf8f3d768a91ecfb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

  Analyse du modèle : HGB...


2026/01/16 17:57:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run model_HGB at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/52b93c0dcef04a4e94696dc40821a3d1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

  Analyse du modèle : XgBoost...


2026/01/16 17:57:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run model_XgBoost at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/36ddc265096a4a6aa9ff354e2d51bd41
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997
🏃 View run benchmark_20260116_175710 at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/50ee3be00e12441db0ac352bf41ad531
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997


In [23]:
# Création du tableau comparatif des modèles
print("\n" + "="*150)
print("RÉSULTATS DE COMPARAISON DES MODÈLES")
print("="*150)
display(results_df)
print("="*150)


RÉSULTATS DE COMPARAISON DES MODÈLES


,Model,R2_Train,R2_Test,CV_R2_Mean,CV_R2_Std,RMSE_Train,RMSE_Test,CV_RMSE_Mean,MAE_Train,MAE_Test,CV_MAE_Mean,Overfit
2,RF,0.998727,0.950765,0.667423,0.140116,2935.423338,21182.480086,45220.408438,915.933461,10651.843921,25235.458844,0.047962
4,XgBoost,0.985921,0.950386,0.727602,0.113506,9762.469030,21263.809769,41199.634821,6102.463806,12319.883681,24068.981217,0.035535
3,HGB,0.967718,0.933805,0.718596,0.126794,14782.714802,24561.294961,41721.818866,9042.238248,14593.325171,23965.739762,0.033913
1,Ridge,0.762880,0.725296,0.602139,0.092263,40064.410430,50034.824976,51022.189594,27892.401399,33355.235455,32793.653103,0.037584
0,Dummy,0.000000,-0.021410,-0.007882,0.010104,82276.215796,96480.528976,82143.083776,62234.701261,69103.071837,62328.887754,0.021410


En comparant les 5 modèles nous constatons que Random Forest, Xgboost et Hist Gradient Boost se distinguent par leur performances.

- RF  :
    - RF affiche un R2 de 0,66 en cross-validation, puis 0.99 en phase de train et 0.95 en phase de test. Le modèle explique 95.08% de la variance du rendement.

    - Cela voudrait dire que pendant la validation croisée la performance du modèle est moyenne, puis la qualité s'améliore fortement pendant l'entrainement pour approcher la perfection (presque 1), et baisse ensuite à 0,95 en phase de test.

    - Le modèle apprend exceptionnellement bien pendant l'entrainement mais ne généralise pas de la même manière. Ceci indique que le modèle est en surapprentissage. La mesure des erreurs est plus faible en Train que en Test.

    - L'erreur moyenne des prédictions est de ~21,000 hg/ha. Si le rendement réel d'un agriculteur pour une culture est de 40,000 hg/ha, la prédiction sera entre 21,000 et 61,000 hg/ha.
    
    - En moyenne, la prédiction s'écarte de ~10,652 hg/ha de la réalité. Ce qui acceptable.

    - Globalment, la performance de ce modèle reste très bonne malgé le léger sur-aprentissage.
 
 
- XGBoost :
    - Le modèle affiche un R2 de 0,73 en CV, puis 0,98 en train et 0,95 en Test. Le modèle explique 95 % de la variance du rendement, ce qui atteste de sa bonne capacité de généralisation. La performance du modèle est très bonne pendant la validation croisée, et également en Train et Test, avec moins d'overfitting comparé au modèle RF.

    - L'erreur moyenne de prédiction est presque similaire à celle du RF soit environ 21,000 hg/ha, et la prédiction s'écarte en moyenne de 12319 hg/ha de la réalité. Les erreurs du modèles sont acceptables, ainsi que l'overfitting qui est moins élevé que pour RF.

    - XGBoost a des performance presque similaires à celle du RF, avec un overfitting plus léger, mais une moyenne d'erreur de prédiction de rendement plus élevée.

- Hist Gradient Boost : 
    - Le modèle affiche un R2 de 0,72 en CV, 0,97 en train et 0,93 en test. Le modèle explique 93% de la variance des rendements.
    La performance de ce modèle est également très bonne, mais celle du XGBoost et RF restent meilleures.

    - Les erreurs du modèle sont acceptables mais plus élevées que celle du RF et Xgboost, avec un overfitting plus bas que pour RF.

 -  Nous retenons que Random Forest et XGBoost sont les deux modèles qui se distinguent par leurs bonnes performances. Nous allons maintenant procéder à une optimisation des hyperparamètres pour améliorer encore plus leur performance.



-------------------------------
## **Optimisation**
-------------------------------

In [28]:
# Grille d'hyperparamètres selon le modèle

def get_param_grid(model_name: str) -> dict:
    if model_name == "RF":
        return {
            "model__n_estimators": [300, 600, 1000],
            "model__max_depth": [10, 20, 30, None],
            "model__min_samples_leaf": [1, 3, 5, 10, 20],
            "model__min_samples_split": [2, 5, 10, 20],
            "model__max_features": ["sqrt", "log2", 0.5, 0.8],
            "model__bootstrap": [True, False],
        }

    elif model_name == "XgBoost":
        return {
            "model__n_estimators": [500, 1000, 2000],
            "model__learning_rate": [0.01, 0.05, 0.1],
            "model__max_depth": [3, 4, 6, 8],
            "model__min_child_weight": [1, 3, 5, 10],
            "model__subsample": [0.6, 0.8, 1.0],
            "model__colsample_bytree": [0.6, 0.8, 1.0],
            "model__gamma": [0, 0.1, 0.5, 1.0],
            "model__reg_alpha": [0, 0.1, 1.0],
            "model__reg_lambda": [1.0, 5.0, 10.0],
        }

    elif model_name == "HGB":
        return {
            "model__learning_rate": [0.01, 0.05, 0.1],
            "model__max_iter": [300, 600, 1200],
            "model__max_depth": [3, 5, 8, None],
            "model__min_samples_leaf": [5, 10, 20, 50],
            "model__l2_regularization": [0, 0.1, 1.0, 5.0, 10.0],
            "model__max_bins": [128, 255],
        }

    else:
        raise ValueError(f"Modèle non supporté : {model_name}")

In [29]:
with mlflow.start_run(run_name=f"optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as parent_opt:

    print("\n OPTIMISATION DES MEILLEURS MODÈLES")
    top_models = results_df.head(3)["Model"].tolist()
    print("✓ Modèles à optimiser :", top_models)

    optimized_results = []
    optimized_models = {}

    for model_name in top_models:
        
        print(f"\n========== OPTIMISATION : {model_name} ==========")

        base_pipeline = models[model_name]  

        # Grille d’hyperparamètres selon le modèle
        param_grid = get_param_grid(model_name)

        print(f"✓ Lancement de RandomizedSearchCV...")
        print(f"  Nombre d'itérations:", CFG.n_iter_search)

        # RandomizedSearchCV

        random_search = RandomizedSearchCV(
            estimator=base_pipeline, 
            param_distributions=param_grid,
            n_iter=CFG.n_iter_search, #  nombre d’essais (clé du gain de temps)
            cv=CFG.cv_folds, # 5 folds
            scoring='r2', n_jobs=-1, verbose=1, random_state=SEED
            )
        
        # Entrainement
        random_search.fit(X_train, y_train)
    
        # Meilleurs paramètres
        best_params = random_search.best_params_
        print(f"\n✓ Meilleurs hyperparamètres trouvés:")

        optimized_model = random_search.best_estimator_

        with mlflow.start_run(run_name=f"Optimized_{model_name}", nested=True):
            
            # Log des paramètres optimisés
            for param, value in random_search.best_params_.items():
                print(f"  {param}: {value}")
                mlflow.log_param(f"best_{param}", value)
    
            #Modèle optimisé
            optimized_model = random_search.best_estimator_
    
            # Prédictions
            y_pred_train_opt = optimized_model.predict(X_train)
            y_pred_test_opt = optimized_model.predict(X_test)
    
            # Métriques finales
            
            r2_train_opt = r2_score(y_train, y_pred_train_opt)
            r2_test_opt = r2_score(y_test, y_pred_test_opt)

            rmse_train_opt = np.sqrt(mean_squared_error(y_train, y_pred_train_opt))
            rmse_test_opt = np.sqrt(mean_squared_error(y_test, y_pred_test_opt))

            mae_train_opt = mean_absolute_error(y_train, y_pred_train_opt)
            mae_test_opt = mean_absolute_error(y_test, y_pred_test_opt)

            # Calcul de l'Overfit = différence R² train/test
            overfit_opt = r2_train_opt - r2_test_opt

            # Log des métriques
            mlflow.log_metrics({
                "R2_train": r2_train_opt,
                "R2_test": r2_test_opt,
                "RMSE_train": rmse_train_opt,
                "RMSE_test": rmse_test_opt,
                "MAE_train": mae_train_opt,
                "MAE_test": mae_test_opt,
                "Overfit": overfit_opt
                })

            # Log CV results
            cv_df = pd.DataFrame(random_search.cv_results_)
            cv_path = f"artifacts/cv_results_{model_name}.csv"
            cv_df.to_csv(cv_path, index=False)
            mlflow.log_artifact(cv_path)
            mlflow.log_metric("CV_best_score", random_search.best_score_)

            # diagnostics visuels - Artefacts
            os.makedirs("artifacts", exist_ok=True)
            rv_path = f"artifacts/residuals_opt_{model_name}.png"
            pv_path = f"artifacts/pred_vs_true_opt_{model_name}.png"
            save_residual_plot(y_test.values, y_pred_test_opt, rv_path)
            save_pred_vs_true_plot(y_test.values, y_pred_test_opt, pv_path)
            mlflow.log_artifact(rv_path)
            mlflow.log_artifact(pv_path)

            # Affichage des métriques : 
            print(f"\n✓ RÉSULTATS DU MODÈLE OPTIMISÉ:")
            print(f"R² Train   : {r2_train_opt:.4f}")
            print(f"R² Test    : {r2_test_opt:.4f}")
            print(f"RMSE Train : {rmse_train_opt:.4f}")
            print(f"RMSE Test  : {rmse_test_opt:.4f}")
            print(f"MAE Train   : {mae_train_opt:.4f}")
            print(f"MAE Test   : {mae_test_opt:.4f}")

            optimized_results.append({
                "Model": model_name,
                "R2_train_opt" : r2_train_opt,
                "R2_test_opt": r2_test_opt,
                "RMSE_train_opt" : rmse_train_opt,
                "RMSE_test_opt": rmse_test_opt,
                "MAE_train_opt": mae_train_opt,
                "MAE_test_opt": mae_test_opt,
                'Overfit': overfit_opt,
                "Best_Params": best_params
                })

            optimized_models[model_name] = optimized_model
            # Log du modèle optimisé
            mlflow.sklearn.log_model(optimized_model, "optimized_model")

              # Tags métier
            mlflow.set_tag("business_goal", "maximize_yield_and_economic_return")
            mlflow.set_tag("target", CFG.target)

            optimized_results_df = pd.DataFrame(optimized_results)
            optimized_results_df = optimized_results_df.sort_values("R2_test_opt", ascending=False)




 OPTIMISATION DES MEILLEURS MODÈLES
✓ Modèles à optimiser : ['RF', 'XgBoost', 'HGB']

========== OPTIMISATION : RF ==========
✓ Lancement de RandomizedSearchCV...
  Nombre d'itérations: 50
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✓ Meilleurs hyperparamètres trouvés:
  model__n_estimators: 1000
  model__min_samples_split: 20
  model__min_samples_leaf: 1
  model__max_features: sqrt
  model__max_depth: None
  model__bootstrap: True


2026/01/16 22:13:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



✓ RÉSULTATS DU MODÈLE OPTIMISÉ:
R² Train   : 0.9833
R² Test    : 0.9337
RMSE Train : 10639.1255
RMSE Test  : 24577.8615
MAE Train   : 4999.4925
MAE Test   : 13586.3844
🏃 View run Optimized_RF at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/36071421178f4128a430c69992b873cd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

========== OPTIMISATION : XgBoost ==========
✓ Lancement de RandomizedSearchCV...
  Nombre d'itérations: 50
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✓ Meilleurs hyperparamètres trouvés:
  model__subsample: 0.6
  model__reg_lambda: 10.0
  model__reg_alpha: 0.1
  model__n_estimators: 2000
  model__min_child_weight: 1
  model__max_depth: 4
  model__learning_rate: 0.05
  model__gamma: 0.5
  model__colsample_bytree: 0.8


2026/01/16 22:14:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



✓ RÉSULTATS DU MODÈLE OPTIMISÉ:
R² Train   : 0.9822
R² Test    : 0.9463
RMSE Train : 10963.7393
RMSE Test  : 22113.0633
MAE Train   : 7033.9058
MAE Test   : 13215.1716
🏃 View run Optimized_XgBoost at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/d73c7026d98c4263a4a4916aa4c2574e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

========== OPTIMISATION : HGB ==========
✓ Lancement de RandomizedSearchCV...
  Nombre d'itérations: 50
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✓ Meilleurs hyperparamètres trouvés:
  model__min_samples_leaf: 5
  model__max_iter: 600
  model__max_depth: None
  model__max_bins: 255
  model__learning_rate: 0.1
  model__l2_regularization: 1.0


2026/01/16 22:17:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



✓ RÉSULTATS DU MODÈLE OPTIMISÉ:
R² Train   : 0.9943
R² Test    : 0.9576
RMSE Train : 6211.9696
RMSE Test  : 19665.0725
MAE Train   : 3661.8908
MAE Test   : 10714.7354
🏃 View run Optimized_HGB at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/cbfa50f6e1a4414590312c2533f9dd65
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997
🏃 View run optimization_20260116_220816 at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/b3996eb1aec0402881415bf5ac686007
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997


In [13]:
optimized_results_df

,Model,R2_train_opt,R2_test_opt,RMSE_train_opt,RMSE_test_opt,MAE_train_opt,MAE_test_opt,Overfit,Best_Params
2,HGB,0.994300,0.957566,6211.969607,19665.072457,3661.890820,10714.735401,0.036733,"{'model__min_samples_leaf': 5, 'model__max_ite..."
1,XgBoost,0.982243,0.946344,10963.739315,22113.063284,7033.905814,13215.171607,0.035899,"{'model__subsample': 0.6, 'model__reg_lambda':..."
0,RF,0.983279,0.933716,10639.125466,24577.861537,4999.492478,13586.384354,0.049563,"{'model__n_estimators': 1000, 'model__min_samp..."


En comparant les 3 meilleurs modèles optimisés nous retenons les constats suivants :

- Pour Random Forest : 
    -  Légère baisse de l'overfiting ( on passe de 0.047 à 0.036 )
    -  Légère baisse du R2 ( train = 0.983 vs 0.998 et test = 0.933 vs 0.950  )
    - RMSE et MAE plus élevées pour le RF optimisé.


- Pour XGBoost : 
    - Overfitting identique (0.035)
    - R2 subtilement moins élevé (0.946 vs 0.950 en test)
    - RMSE et MAE plus élevées.


- Pour HGB : 
    - Overfitting légèrement plus bas ( 0.036 vs 0.033 )
    - R2 amélioré en train et en test ( on passe d'un R2 de 0.950 avant optimisation à 0.957 en test )
    - RMSE et MAE moins élevées. ( RMSE = 19665, et MAE= 10714 )
    

**Les 3 modèles ont des performances légèrement différentes, mais HGB se distingue, avec moins de sur-aprentissage, et des erreurs moins élévées. Dans un contexte métier de prédiction de rendement en agriculture, nous avons besoin de minimiser les erreurs moyennes des prédictions fournies au agriculteur. Nous allons donc retenir comme model le HIST GRADIENT BOOST optimisé, dont les valeurs prédites ne s'écarte en moyenne que de 10714 hg/ha.** 

In [30]:
# ================== IMPORTANCE DES VARIABLES ==================
print("\n ANALYSE DE L'IMPORTANCE DES VARIABLES")

best_hgb = optimized_models["HGB"]

perm = permutation_importance(
    estimator=best_hgb,          # pipeline complet
    X=X_test,                    # X brut avant preprocessing
    y=y_test,                    # y brut
    scoring="neg_mean_absolute_error",
    n_repeats=10,
    random_state=SEED,
    n_jobs=-1
    )

perm_df = pd.DataFrame({
    "feature": X_test.columns,   # les colonnes brutes avant preprocessing
    "importance_mean": perm.importances_mean,
    "importance_std": perm.importances_std
}).sort_values("importance_mean", ascending=False)
 
# Log de l'importance dans MLflow

with mlflow.start_run(run_name="feature_importance_HGB", nested=True):

    # Tags
    mlflow.set_tag("model_name", "HGB")
    mlflow.set_tag("interpretability_method", "permutation_importance")
    mlflow.set_tag("scoring", "neg_mean_absolute_error")

    # Sauvegarde CSV
    os.makedirs("artifacts", exist_ok=True)
    perm_imp_path = "artifacts/permutation_importance_HGB.csv"
    perm_df.to_csv(perm_imp_path, index=False)
    mlflow.log_artifact(perm_imp_path)

    # Log top-k importances comme métriques (optionnel mais excellent)
    for i, row in perm_df.head(10).iterrows():
        mlflow.log_metric(f"FI_{row['feature']}", row["importance_mean"])

print("\n" + "="*60)
print("IMPORTANCE DES VARIABLES")
print("="*60)
display(perm_df)
print("="*60)



 ANALYSE DE L'IMPORTANCE DES VARIABLES
🏃 View run feature_importance_HGB at: http://127.0.0.1:5000/#/experiments/765168008382645997/runs/561159a8f6184a1a904d812da45f76d8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/765168008382645997

IMPORTANCE DES VARIABLES


,feature,importance_mean,importance_std
1,item,71188.856808,821.364324
0,area,15450.016641,327.273959
5,avg_temp,13184.805172,195.977642
4,pesticides_tonnes,8319.583735,259.738316
3,avg_rain_mm,6305.237289,125.287516
2,year,0.000000,0.000000


En Analysant la Permutation Feature importance nous constatons que le premier facteur impactant le rendement c'est le type de culture, puis le pays de culture, et ensuite des températures dans le pays de culture, l'usage de pesticide et la pluie.

-------------------------------
## **Sauvegarde du Modèle**
-------------------------------

In [16]:
# chemin du modèle
model_path = MODEL_DIR / "hgb_optimized.joblib"

# sauvegarde
joblib.dump(best_hgb, model_path)

print(f"✅ Modèle sauvegardé ici : {model_path}")

✅ Modèle sauvegardé ici : /Users/fatiza/Documents/DATA SCIENTIST/Projet12/OC/p12/model/hgb_optimized.joblib


In [19]:
#================================================================================
# Fonction pour ajouter l'effet de l'irrigation et fertilization sur le rendement 
#================================================================================

def apply_optional_scenarios(yield_hg_ha: float, irrigation: bool = False, fertilizer: bool = False) -> float:
    """
    Post-ajustement additif (what-if) pour irrigation/fertilisation.
    Hypothèse: effet moyen constant, additif.
    """
    adj = 0.0
    if irrigation:
        adj += 12000
    if fertilizer:
        adj += 15000
    return yield_hg_ha + adj

#================================================================================
# Fonction pour ajuster le prix vs unité de rendement
#================================================================================

def compute_revenue_per_ha(yield_hg_ha: float, price_value: float, price_unit: str = "eur_per_t") -> float:
    """
    Convertit un rendement (hg/ha) en revenu/ha selon une unité de prix.
    - eur_per_t : €/tonne  -> revenue = yield_hg_ha * price/10_000
    - eur_per_kg: €/kg     -> revenue = yield_hg_ha * price/10
    - eur_per_hg: €/hg     -> revenue = yield_hg_ha * price
    """
    u = price_unit.lower().strip()
    if u in ["eur_per_t", "€/t", "euro_per_tonne"]:
        return yield_hg_ha * (price_value / 10_000)
    if u in ["eur_per_kg", "€/kg", "euro_per_kg"]:
        return yield_hg_ha * (price_value / 10)
    if u in ["eur_per_hg", "€/hg", "euro_per_hg"]:
        return yield_hg_ha * price_value
    raise ValueError(f"Unsupported price_unit: {price_unit}")

In [20]:
model = best_hgb

# ========================================================
# Moteur de Prediction + 2 recommenders (yield vs revenue)
# ========================================================
def predict_yield_hg_ha(
    model, *,
    area: str, item: str, year: int,
    avg_rain_mm: float, pesticides_tonnes: float, avg_temp: float,
    irrigation: bool = False, fertilizer: bool = False ) -> float:
    X_in = pd.DataFrame([{
        "area": area,
        "item": item,
        "year": year,
        "avg_rain_mm": avg_rain_mm,
        "pesticides_tonnes": pesticides_tonnes,
        "avg_temp": avg_temp
        }])
    base_pred = float(model.predict(X_in)[0])
    return apply_optional_scenarios(base_pred, irrigation=irrigation, fertilizer=fertilizer)

# ========================================================
# Moteur de Recommendation ( Hg/ha yield & rentabilité)
# ========================================================
def recommend_by_yield(
    model, *,
    area: str, year: int,
    avg_rain_mm: float, pesticides_tonnes: float, avg_temp: float,
    candidate_items: list[str],
    irrigation: bool = False, fertilizer: bool = False,
    top_k: int = 5) -> pd.DataFrame:
    X_in = pd.DataFrame([{
        "area": area,
        "item": it,
        "year": year,
        "avg_rain_mm": avg_rain_mm,
        "pesticides_tonnes": pesticides_tonnes,
        "avg_temp": avg_temp
    } for it in candidate_items])

    base_preds = model.predict(X_in).astype(float)
    adj = (CFG.COEF_IRRIGATION_HG_HA if irrigation else 0.0) + (CFG.COEF_FERTILIZATION_HG_HA if fertilizer else 0.0)
    preds = base_preds + adj

    out = pd.DataFrame({
        "item": candidate_items,
        "pred_yield_hg_ha": preds,
        "pred_yield_t_ha": preds / 10000,
        "irrigation": irrigation,
        "fertilizer": fertilizer
    }).sort_values("pred_yield_hg_ha", ascending=False)

    return out.head(top_k).reset_index(drop=True)

def recommend_by_revenue(
    model, *,
    area: str, year: int,
    avg_rain_mm: float, pesticides_tonnes: float, avg_temp: float,
    candidate_items: list[str],
    prices: dict[str, float],
    price_unit: str = "eur_per_t",
    irrigation: bool = False, fertilizer: bool = False,
    top_k: int = 5
) -> pd.DataFrame:
    # garder uniquement les items dont l'agriculteur a fourni le prix
    items = [it for it in candidate_items if it in prices]
    if len(items) == 0:
        raise ValueError("No candidate items have a provided price. Provide prices like {'maize': 180, ...}.")

    X_in = pd.DataFrame([{
        "area": area,
        "item": it,
        "year": year,
        "avg_rain_mm": avg_rain_mm,
        "pesticides_tonnes": pesticides_tonnes,
        "avg_temp": avg_temp
    } for it in items])

    base_preds = model.predict(X_in).astype(float)
    adj = (CFG.COEF_IRRIGATION_HG_HA if irrigation else 0.0) + (CFG.COEF_FERTILIZATION_HG_HA if fertilizer else 0.0)
    preds = base_preds + adj

    out = pd.DataFrame({
        "item": items,
        "pred_yield_hg_ha": preds,
        "pred_yield_t_ha": preds / 10_000,
        "price_value": [prices[it] for it in items],
        "price_unit": price_unit,
        "irrigation": irrigation,
        "fertilizer": fertilizer
    })

    out["revenue_per_ha"] = out.apply(
        lambda r: compute_revenue_per_ha(r["pred_yield_hg_ha"], r["price_value"], r["price_unit"]),
        axis=1)

    out = out.sort_values("revenue_per_ha", ascending=False)
    return out.head(top_k).reset_index(drop=True)

In [26]:
 
# Exemple d'entrée pour prédiction simple
area = "belgium"
item = "potatoes"
year = 2020
avg_rain_mm = 1200.0
pesticides_tonnes = 100.0
avg_temp = 16.0


pred_yield = predict_yield_hg_ha(
    model=best_hgb,
    area=area,
    item=item,
    year=year,
    avg_rain_mm=avg_rain_mm,
    pesticides_tonnes=pesticides_tonnes,
    avg_temp=avg_temp,
    irrigation=True,       # teste l’effet irrigation
    fertilizer=True        # teste l’effet fertilisation
)

print(f"Rendement prévu (hg/ha) pour {item}: {pred_yield:.2f}")


Rendement prévu (hg/ha) pour potatoes: 275495.24


In [30]:
candidate_items = ["maize", "potatoes", "rice, paddy", "soybeans", "sorghum"]

yield_ranking = recommend_by_yield(
    model=best_hgb,
    area=area,
    year=year,
    avg_rain_mm=avg_rain_mm,
    pesticides_tonnes=pesticides_tonnes,
    avg_temp=avg_temp,
    candidate_items=candidate_items,
    irrigation=False,
    fertilizer=True,
    top_k=5
)

display(yield_ranking)


,item,pred_yield_hg_ha,pred_yield_t_ha,irrigation,fertilizer
0,potatoes,263495.235949,26.349524,False,True
1,"rice, paddy",107571.822978,10.757182,False,True
2,maize,98545.575830,9.854558,False,True
3,sorghum,87343.681251,8.734368,False,True
4,soybeans,78064.689589,7.806469,False,True


In [33]:
# Prix fictifs €/t pour chaque culture
prices = {
    "maize": 180,
    "potatoes": 50,
    "rice, paddy": 220,
    "soybeans": 300,
    "sorghum": 200
}

revenue_ranking = recommend_by_revenue(
    model=best_hgb,
    area=area,
    year=year,
    avg_rain_mm=avg_rain_mm,
    pesticides_tonnes=pesticides_tonnes,
    avg_temp=avg_temp,
    candidate_items=candidate_items,
    prices=prices,
    price_unit="eur_per_t",
    irrigation=True,
    fertilizer=True,
    top_k=5
)

display(revenue_ranking)


,item,pred_yield_hg_ha,pred_yield_t_ha,price_value,price_unit,irrigation,fertilizer,revenue_per_ha
0,soybeans,90064.689589,9.006469,300,eur_per_t,True,True,2701.940688
1,"rice, paddy",119571.822978,11.957182,220,eur_per_t,True,True,2630.580106
2,maize,110545.575830,11.054558,180,eur_per_t,True,True,1989.820365
3,sorghum,99343.681251,9.934368,200,eur_per_t,True,True,1986.873625
4,potatoes,275495.235949,27.549524,50,eur_per_t,True,True,1377.476180
